In [ ]:
import numpy as np
import scipy
from scipy.optimize import minimize
from tqdm import tqdm, trange
import matplotlib.pyplot as plt

In [ ]:
SEED = 0

In [ ]:
class IdentityQuantizer:
    def quantize(xs, rng):
        return xs
    
    @staticmethod
    def homogeneous_ab(n, d):
        return 0, 0
    
    @staticmethod
    def bits_per_client(n, d):
        return 32 * d

class CorrelatedQuantizer:
    def quantize(xs, rng):
        n, d = xs.shape
        r = np.linalg.norm(xs, axis=1).max() # 32 bits per client for norm
        
        xs_normalized = (xs + r) / (2 * r)
        
        gammas = rng.random((n, d)) / n # generate gammas in [0, 1/n)
        perms = rng.permuted(np.tile(np.linspace(0, 1, n, endpoint=False), d).reshape(d, n).T, axis=0)
        thresholds = perms + gammas
        
        xs_compressed = np.zeros_like(xs)
        xs_compressed[xs_normalized > thresholds] = 1 # 1 bit per coordinate per client
        
        xs_decompressed = 2 * r * xs_compressed - r
        return xs_decompressed
    
    @staticmethod
    def homogeneous_ab(n, d):
        return d / 4 / n**2, 0
    
    @staticmethod
    def bits_per_client(n, d):
        return 32 + d


class UncorrelatedQuantizer:
    @staticmethod
    def quantize(xs, rng):
        n, d = xs.shape
        r = np.linalg.norm(xs, axis=1)[:, np.newaxis]
        
        xs_normalized = (xs + r) / (2 * r)
        
        thresholds = rng.random((n, d))
        
        xs_compressed = np.zeros_like(xs)
        xs_compressed[xs_normalized > thresholds] = 1
        
        xs_decompressed = 2 * r * xs_compressed - r
        return xs_decompressed
    
    @staticmethod
    def homogeneous_ab(n, d):
        return d / 4 / n, 0
    
    @staticmethod
    def bits_per_client(n, d):
        return 32 + d


class RandK:
    @staticmethod
    def quantize(xs, rng):
        n, d = xs.shape
    
        k = int(0.1 * d)
        # Sample a random subset of coordinates without replacement
        idx = rng.random(xs.shape).argsort(1)[:,:k]
        
        xs_decompressed = np.zeros_like(xs)
        np.put_along_axis(xs_decompressed, idx, np.take_along_axis(xs, idx, axis=1), axis=1)
        return xs_decompressed * (d / k)
    
    @staticmethod
    def homogeneous_ab(n, d):
        k = int(0.1 * d)
        return (d / k - 1) / n, 0
    
    @staticmethod
    def bits_per_client(n, d):
        k = int(0.1 * d)
        return 32 * k


class PermK:
    def quantize(xs, rng):
        n, d = xs.shape
        if n % d == 0:
            q = n // d
            
            idx = rng.permuted(np.arange(n)).reshape(d, q)
            
            xs_decompressed_t = np.zeros_like(xs).T
            np.put_along_axis(xs_decompressed_t, idx, np.take_along_axis(xs.T, idx, axis=1), axis=1)
            return xs_decompressed_t.T * d
        elif d % n ==0:
            q = d // n
            idx = rng.permuted(np.arange(d)).reshape(n, q)
            
            xs_decompressed = np.zeros_like(xs)
            np.put_along_axis(xs_decompressed, idx, np.take_along_axis(xs, idx, axis=1), axis=1)
            return xs_decompressed * n
        else:
            raise ValueError("n must be a multiple of d or vice versa")
    
    @staticmethod
    def homogeneous_ab(n, d):
        if d > n:
            return 1, 1
        else:
            return 1 - (n - d) / (n - 1), 1 - (n - d) / (n - 1)
        
    @staticmethod
    def bits_per_client(n, d):
        if d > n:
            return 32 * d/n
        else:
            return 32


class Natural:
    def quantize(xs, rng: np.random.Generator):
        n, d = xs.shape

        zeros_pos = (xs == 0)
        xs[zeros_pos] = 1
        
        alpha = np.log2(abs(xs))
        abs_down = np.power(2, np.floor(alpha))
        abs_up = np.power(2, np.ceil(alpha))
        pt = (abs_up - abs(xs)) / abs_down
        sample = rng.binomial(1, pt).astype(bool)
        
        out = np.zeros_like(xs)
        np.putmask(out, sample, abs_down)
        np.putmask(out, ~sample, abs_up)
        np.putmask(out, zeros_pos, 0)
        out *= np.sign(xs)
        return out
    
    @staticmethod
    def homogeneous_ab(n, d):
        return 1 / 8 / n, 0
    
    @staticmethod
    def bits_per_client(n, d):
        return 9 * d
    
    
    
def hadamard_transform_(vec):
    """fast Walsh–Hadamard transform (in-place)

    :param vec: vec is expected to be a power of 2!
    :return: the Hadamard transform of vec
    """
    n = vec.shape[0]
    d = vec.shape[1]
    h = 2
    while h <= d:
        hf = h // 2
        vec = vec.reshape(n, d // h, h)

        ## the following is a more inplace way of doing the following:
        # half_1 = batch[:, :, :hf]
        # half_2 = batch[:, :, hf:]
        # batch = torch.cat((half_1 + half_2, half_1 - half_2), dim=-1)
        # the NOT inplace seems to be actually be slightly faster
        # (I assume for making more memory-contiguous operations. That being said,
        # it more easily throws out-of-memory and may slow things overall,
        # so using inplace version below:)

        vec[:, :, :hf] = vec[:, :, :hf] + vec[:, :, hf : 2 * hf]
        vec[:, :, hf : 2 * hf] = vec[:, :, :hf] - 2 * vec[:, :, hf : 2 * hf]
        h *= 2

    vec *= d**-0.5  # vec /= np.sqrt(d)

    return vec.reshape(n, d)


class DriveQuantizer:
    def quantize(xs, rng: np.random.Generator):
        n, d = xs.shape
        
        # Pad
        next_power_of_2 = 1 << (d - 1).bit_length()
        xs = np.pad(xs, ((0, 0), (0, next_power_of_2 - d)))
        
        # Rotate

        rademacher = 2 * rng.binomial(1, 0.5, size=xs.shape).astype(xs.dtype) - 1
        xs = hadamard_transform_(xs * rademacher)

        # Quantize
        normalized_data = xs * np.sqrt(d) / np.linalg.norm(xs, axis=1)[:, np.newaxis]

        quantized_xs = np.zeros_like(xs)
        quantized_xs[normalized_data > 0] = np.sqrt(2/np.pi)
        quantized_xs[normalized_data < 0] = -np.sqrt(2/np.pi)

        # Dequantize
        dequantized_xs = (np.linalg.norm(xs, axis=1) ** 2 / np.sum(xs * quantized_xs, axis=1))[:, np.newaxis] * quantized_xs
        np.nan_to_num(dequantized_xs, copy=False)
        
        # Unrotate
        out = hadamard_transform_(dequantized_xs) * rademacher
        
        # Unpad
        out = out[:,:d]
        return out
    
    @staticmethod
    def homogeneous_ab(n, d):
        return (np.pi / 2 - 1) / n, 0
    
    @staticmethod
    def bits_per_client(n, d):
        next_power_of_2 = 1 << (d - 1).bit_length()
        return 32 + next_power_of_2
    

In [ ]:
def marina_client_step(i: int, broadcasted_grad, prev_grads, functions, p: float, gamma: float, quantize_fn, rng):
    functions.apply_update(- gamma * broadcasted_grad) 
    grads, values = functions.get_grad_and_values()
    c = i == 0 or rng.random() < p
    if c:
        uncompressed_updates = grads
        compressed_updates = uncompressed_updates
    else:
        uncompressed_updates = grads - prev_grads
        compressed_updates = quantize_fn(uncompressed_updates, rng)
    return grads, values, c, uncompressed_updates, compressed_updates


def marina_master_step(updates, prev_grad, c):
    if c:
        prev_grad = updates.mean(axis=0)
    else:
        prev_grad = prev_grad + updates.mean(axis=0)
    return prev_grad, updates.shape[0]


def weighted_marina_master_step(updates, prev_grad, c, rng, batch_size, probs):
    n = updates.shape[0]
    if c:
        prev_grad = updates.mean(axis=0)
        num_unique_clients = n
    else:
        batch_idx = np.random.choice(n, p=probs, size=batch_size)
        num_unique_clients = len(np.unique(batch_idx))
        sampled_client_updates = updates[batch_idx,:] / probs[batch_idx] / n
        prev_grad = prev_grad + sampled_client_updates.mean(axis=0)
    return prev_grad, num_unique_clients

def marina_homogeneous_p(a, b, d, bits_per_client, liptschitz_constants):
    assert b == 0, "Not implemented"
    mean_liptschitz_gradient_constant = sum(liptschitz_constants) / len(liptschitz_constants)
    mean_square_liptschitz_gradient_constant = (sum(l**2 for l in liptschitz_constants) / len(liptschitz_constants)) ** (1 / 2)
    a *= (mean_square_liptschitz_gradient_constant / mean_liptschitz_gradient_constant) ** 2
    
    c = bits_per_client / (32 * d)
    
    def func_(p):
        return (p + (1 - p) * c) * (1 + np.sqrt((1 - p) / p * a))

    def grad_(p):
        return ((-a / p - a * (1 - p) / p**2) * (p + c * (1 - p))) / (2 * np.sqrt(a * (1 - p) / p)) + (1 - c) * (
            1 + np.sqrt(a * (1 - p) / p)
        )

    result = minimize(func_, 0.5, method="SLSQP", jac=grad_, bounds=[(1e-5, 1 - 1e-5)], tol=1e-10)
    assert result.success, result.message
    return result.x[0]
    

def marina_homogeneous_gamma(a, b, liptschitz_constants, smoothness_variance, p, num_clients):
    mean_liptschitz_gradient_constant = sum(liptschitz_constants) / num_clients
    mean_square_liptschitz_gradient_constant = (sum(l**2 for l in liptschitz_constants) / num_clients) ** (1 / 2)
    
    if b == 0:
        smoothness_variance = 0 # Doesn't matter
    if a == 0:
        p = 1 # Doesn't matter
    m = mean_liptschitz_gradient_constant + np.sqrt(
        ((1 - p) / p) * ((a - b) * mean_square_liptschitz_gradient_constant**2 + b * smoothness_variance**2)
    )
    
    return 1/m

In [ ]:
def measure(functions, quantizer, gamma_mult, rng, p=None, steps=None, comm_limit=None, use_importance_sampling=False, batch_size=None):
    assert (steps is None) ^ (comm_limit is None), "Either steps or comm_limit must be specified"
    if use_importance_sampling:
        assert batch_size is not None, "Batch size must be specified for importance sampling"
    
    n, d = functions.get_nd()
    
    a, b = quantizer.homogeneous_ab(n, d)
    liptschitz_constants = np.array(functions.get_liptschitz_constants)
    
    if p is None:
        p = marina_homogeneous_p(a, b, d, quantizer.bits_per_client(n, d), liptschitz_constants)
        
    gamma_base = marina_homogeneous_gamma(
        a,
        b,
        liptschitz_constants,
        functions.get_smoothness_variance(),
        p,
        NUM_CLIENTS,
    )
    
    gamma = gamma_mult * gamma_base

    metrics = {
        'values': [],
        'grad_norms_2': [],
        'total_bits_received': [],
        'is_full': [],
        'r': [],
        'sigma': [],
        'updates_mean_squares': [],
        'sigma_r_norm': [],
        'step_size': gamma,
        'step_size_mult': gamma_mult,
        'p': p,
    }
    
    clinets_prev_grads = np.zeros((n, d))
    master_prev_grad = np.zeros((d,))
    
    total_bits_received = 0
    broadcasted_grad = np.zeros((d, ))
    if steps is not None:
        pbar = tqdm(total=steps)
    else:
        pbar = tqdm(total=comm_limit)
        
    step = 0
    while True:
        grads, values, c, uncompressed_updates, compressed_updates = marina_client_step(
            step, 
            broadcasted_grad, 
            clinets_prev_grads, 
            functions,
            p, 
            gamma, 
            quantizer.quantize,
            rng,
        )
        clinets_prev_grads = grads
        
        if use_importance_sampling:
            broadcasted_grad, num_unique_clients = weighted_marina_master_step(
                compressed_updates, 
                master_prev_grad, 
                c, 
                rng, 
                batch_size,
                liptschitz_constants / liptschitz_constants.sum(),
            )
        else:
            broadcasted_grad, num_unique_clients = marina_master_step(compressed_updates, master_prev_grad, c)
        master_prev_grad = broadcasted_grad
        
        # Metrics
        metrics['values'].append(values.mean())
        metrics['grad_norms_2'].append(np.linalg.norm(grads.mean(axis=0))**2)
        metrics['total_bits_received'].append(total_bits_received)
        metrics['is_full'].append(c)
        # metrics['r'].append(np.linalg.norm(uncompressed_updates, axis=1).max())
        # metrics['sigma'].append(np.linalg.norm(uncompressed_updates - uncompressed_updates.mean(axis=0), axis=1).mean())
        # metrics['sigma_r_norm'].append(metrics['sigma'][-1] *  metrics['r'][-1] / (np.linalg.norm(uncompressed_updates, axis=1)**2).mean())
        
        bits_per_client = 32 * d if c else quantizer.bits_per_client(n, d)
        total_bits_received += bits_per_client * num_unique_clients
        step += 1
        
        if steps is not None:
            pbar.update(1)
        else:
            pbar.update(bits_per_client * num_unique_clients)
        
        if steps is not None and step >= steps:
            break
        
        if comm_limit is not None and total_bits_received >= comm_limit:
            break
        
    return metrics

In [ ]:
from sklearn.datasets import load_svmlight_file
from functools import cached_property


NUM_CLIENTS = 64
SEED = 0
RNG = np.random.default_rng(SEED)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class LibSVMFunctions:
    def __init__(self, path, rng, num_clients):
        self.num_clients = num_clients

        X, y = load_svmlight_file(path)
        
        shuffle = rng.permuted(np.arange(y.shape[0]))
        self.X = X[shuffle].todense().astype(np.float32)
        self.y = y[shuffle].astype(np.float32)
        self.X = np.squeeze(np.asarray(self.X))
        self.weight = INIT_WEIGHT.copy()
        
        self.l = 0.1
        
    def reset(self):
        self.weight = INIT_WEIGHT.copy()
        
    def get_nd(self):
        return self.num_clients, len(self.weight)
        
    def get_grad_and_values(self):
        # self.X: batch x features
        # self.weight: features 
         
        numerator = np.multiply(self.X, self.y[:, np.newaxis])
        denominator = 1 + np.exp(np.multiply(self.X @ self.weight, self.y))
        client_grads = np.stack([-client_grad.mean(axis=0) for client_grad in np.array_split(numerator / denominator[:,np.newaxis], self.num_clients)]) + self.l * (2 * self.weight / (1 + self.weight**2)**2)[np.newaxis, :]
            
        l = np.log(1 + np.exp(-self.X.dot(self.weight) * self.y))
        regul_losses = self.l * (self.weight**2 / (1 + self.weight**2)).sum()
        client_losses = np.stack([loss.mean() + regul_losses for loss in np.array_split(l, self.num_clients)])
        
        return client_grads, client_losses
    
    def apply_update(self, update):
        self.weight += update
    
    @cached_property
    def get_liptschitz_constants(self):
        Ls = []
        for client_X in np.array_split(self.X, self.num_clients, axis=0):
            client_hess = (1 / 4 / client_X.shape[0]) * (client_X.T @ client_X) + 2 * self.l * np.eye(len(self.weight))
            L = float(np.max(np.linalg.eigvals(client_hess)))
            Ls.append(L)
        return Ls
    
    def get_smoothness_variance(self):
        return None


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

LINE_KWARGS = {
    "Gradient Descent": {"color": "gray", "linestyle": "solid"},
    "MARINA + CQ": {"color": "green", "linestyle": "dashed"}, 
    "MARINA + IQ":{"color": "red", "linestyle": "dashdot"},
    "MARINA + DRIVE": {"color": "blue", "linestyle": "dotted"},
}
DOT_KWARGS = {
    "Gradient Descent": {"color": "gray"},
    "MARINA + CQ": {"color": "green", "marker": "^"}, 
    "MARINA + IQ":{"color": "red", "linestyle": ":", "marker": "v"}, 
    "MARINA + DRIVE": {"color": "blue", "linestyle": "--", "marker": "o"},
}

def plot_norms_libsvm(ax, metrics, y_key, label, line_kwargs, dot_kwargs):
    full_grad = np.array(metrics["is_full"])
    bits_per_client = np.array(metrics["total_bits_received"]) / NUM_CLIENTS
    grad_norm_2 = np.array(metrics[y_key])
    
    c = ~np.concatenate((full_grad, [True])) & ~np.concatenate(([True], full_grad))
    
    diff = np.diff(c.astype(int))
    starts = np.where(diff == 1)[0]
    ends = np.where(diff == -1)[0]
    
    for start, end in zip(starts, ends):
        ax.plot(bits_per_client[start:end+1], grad_norm_2[start:end+1], label=label, **line_kwargs)
    
    # Connect points inbetween full grads
    for i in range(len(bits_per_client)-1):
        if not full_grad[i] and not full_grad[i+1]:
            pass
            # ax.plot(
            #     bits_per_client[i:i+2],
            #     grad_norm_2[i:i+2],
            #     **dot_kwargs,
            # )
        elif full_grad[i+1]:
            ax.scatter(
                bits_per_client[i],
                grad_norm_2[i],
                **dot_kwargs,
            )
            

def plot_libsvm(all_metrix):
    fig, axs = plt.subplots(1, 3, figsize=(12, 3))
    
    for ax, (dataset, dataset_metrics) in zip(axs, all_metrix.items()):
        for name, metrics in dataset_metrics.items():
            plot_norms_libsvm(
                ax, 
                metrics, 
                "grad_norms_2", 
                f"{name}", 
                LINE_KWARGS[name], 
                DOT_KWARGS[name]
            )

        ax.set_yscale("log")
        ax.set_xlabel("# of bits sent per client")
        ax.set_ylabel(r"$\|\nabla f(x)\|^2$")
        ax.set_title(dataset)
        
        # Legend
        handles = []
        for name in dataset_metrics:
            handle_kwargs = {}
            handle_kwargs.update(DOT_KWARGS[name])
            handle_kwargs.update(LINE_KWARGS[name])
            handle = mlines.Line2D([], [], label=name, **handle_kwargs)
            handles.append(handle)
        ax.legend(handles=handles, loc='upper right')

    fig.tight_layout()
    return fig

In [ ]:
GLOBAL_METRICS = {}

## 4.1 Non-Convex Logistic Regressio

### a9a

In [ ]:
COMM_LIMIT = 3e7
NUM_CLIENTS = 123
INIT_WEIGHT = RNG.normal(size=(NUM_CLIENTS),).astype(np.float32)
TASK = LibSVMFunctions("./data/a9a", RNG, NUM_CLIENTS)
RNG = np.random.default_rng(SEED)

In [ ]:
GD_METRICS = {}
DRIVE_METRICS = {}
CORRELATED_METRICS= {}
UNCORRELATED_METRICS = {}

for lr in [1, 2, 4, 8, 16]:
    TASK.reset()
    GD_METRICS[lr] = measure(
        TASK,
        IdentityQuantizer,
        lr,
        RNG,
        p=0,
        comm_limit=COMM_LIMIT,
    )

    TASK.reset()
    DRIVE_METRICS[lr] = measure(
        TASK,
        DriveQuantizer,
        lr,
        RNG,
        comm_limit=COMM_LIMIT,
    )

    TASK.reset()
    CORRELATED_METRICS[lr] = measure(
        TASK,
        CorrelatedQuantizer,
        lr,
        RNG,
        comm_limit=COMM_LIMIT,
    )

    TASK.reset()
    UNCORRELATED_METRICS[lr] = measure(
        TASK,
        UncorrelatedQuantizer,
        lr,
        RNG,
        comm_limit=COMM_LIMIT,
    )

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(16, 6), sharey=True)

for ax, lr in zip(axs, [1, 2, 4, 8, 16]):
    ax.plot(GD_METRICS[lr]["total_bits_received"], GD_METRICS[lr]["grad_norms_2"], label="GD", **LINE_KWARGS["Gradient Descent"])
    ax.plot(DRIVE_METRICS[lr]["total_bits_received"], DRIVE_METRICS[lr]["grad_norms_2"], label="MARINA + DRIVE", **LINE_KWARGS["MARINA + DRIVE"])
    ax.plot(CORRELATED_METRICS[lr]["total_bits_received"], CORRELATED_METRICS[lr]["grad_norms_2"], label="MARINA + CQ", **LINE_KWARGS["MARINA + CQ"])
    ax.plot(UNCORRELATED_METRICS[lr]["total_bits_received"], UNCORRELATED_METRICS[lr]["grad_norms_2"], label="MARINA + IQ", **LINE_KWARGS["MARINA + IQ"])
    
    ax.set_yscale("log")
    ax.set_title(f"{lr=}")
    ax.legend()
    
fig.tight_layout()
plt.show()

In [ ]:
GLOBAL_METRICS["a9a"] = {
    "Gradient Descent": GD_METRICS[1], 
    "MARINA + DRIVE": DRIVE_METRICS[4],
    "MARINA + CQ": CORRELATED_METRICS[2], 
    "MARINA + IQ": UNCORRELATED_METRICS[4],
}

### w8a

In [ ]:
COMM_LIMIT = 3e8
NUM_CLIENTS = 300
INIT_WEIGHT = RNG.normal(size=(NUM_CLIENTS),).astype(np.float32)
TASK = LibSVMFunctions("./data/w8a", RNG, NUM_CLIENTS)
RNG = np.random.default_rng(SEED)

In [ ]:
GD_METRICS = {}
DRIVE_METRICS = {}
CORRELATED_METRICS= {}
UNCORRELATED_METRICS = {}

for lr in [1, 2, 4, 8, 16]:
    TASK.reset()
    GD_METRICS[lr] = measure(
        TASK,
        IdentityQuantizer,
        lr,
        RNG,
        p=0,
        comm_limit=COMM_LIMIT,
    )

    TASK.reset()
    DRIVE_METRICS[lr] = measure(
        TASK,
        DriveQuantizer,
        lr,
        RNG,
        comm_limit=COMM_LIMIT,
    )

    TASK.reset()
    CORRELATED_METRICS[lr] = measure(
        TASK,
        CorrelatedQuantizer,
        lr,
        RNG,
        comm_limit=COMM_LIMIT,
    )

    TASK.reset()
    UNCORRELATED_METRICS[lr] = measure(
        TASK,
        UncorrelatedQuantizer,
        lr,
        RNG,
        comm_limit=COMM_LIMIT,
    )

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(16, 6), sharey=True)

for ax, lr in zip(axs, [1, 2, 4, 8, 16]):
    ax.plot(GD_METRICS[lr]["total_bits_received"], GD_METRICS[lr]["grad_norms_2"], label="GD", **LINE_KWARGS["Gradient Descent"])
    ax.plot(DRIVE_METRICS[lr]["total_bits_received"], DRIVE_METRICS[lr]["grad_norms_2"], label="MARINA + DRIVE", **LINE_KWARGS["MARINA + DRIVE"])
    ax.plot(CORRELATED_METRICS[lr]["total_bits_received"], CORRELATED_METRICS[lr]["grad_norms_2"], label="MARINA + CQ", **LINE_KWARGS["MARINA + CQ"])
    ax.plot(UNCORRELATED_METRICS[lr]["total_bits_received"], UNCORRELATED_METRICS[lr]["grad_norms_2"], label="MARINA + IQ", **LINE_KWARGS["MARINA + IQ"])
    
    ax.set_yscale("log")
    ax.set_title(f"{lr=}")
    ax.legend()
    
fig.tight_layout()
plt.show()

In [ ]:
GLOBAL_METRICS["madelon"] = {
    "Gradient Descent": GD_METRICS[4], 
    "MARINA + DRIVE": DRIVE_METRICS[2],
    "MARINA + CQ": CORRELATED_METRICS[4], 
    "MARINA + IQ": UNCORRELATED_METRICS[4],
}

### Splice

In [ ]:
COMM_LIMIT = 3e7
NUM_CLIENTS = 60
INIT_WEIGHT = RNG.normal(size=(NUM_CLIENTS),).astype(np.float32)
TASK = LibSVMFunctions("./data/splice", RNG, NUM_CLIENTS)
RNG = np.random.default_rng(SEED)

In [ ]:
GD_METRICS = {}
DRIVE_METRICS = {}
CORRELATED_METRICS= {}
UNCORRELATED_METRICS = {}

for lr in [1, 2, 4, 8, 16]:
    TASK.reset()
    GD_METRICS[lr] = measure(
        TASK,
        IdentityQuantizer,
        lr,
        RNG,
        p=0,
        comm_limit=COMM_LIMIT,
    )

    TASK.reset()
    DRIVE_METRICS[lr] = measure(
        TASK,
        DriveQuantizer,
        lr,
        RNG,
        comm_limit=COMM_LIMIT,
    )

    TASK.reset()
    CORRELATED_METRICS[lr] = measure(
        TASK,
        CorrelatedQuantizer,
        lr,
        RNG,
        comm_limit=COMM_LIMIT,
    )

    TASK.reset()
    UNCORRELATED_METRICS[lr] = measure(
        TASK,
        UncorrelatedQuantizer,
        lr,
        RNG,
        comm_limit=COMM_LIMIT,
    )

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(16, 6), sharey=True)

for ax, lr in zip(axs, [1, 2, 4, 8, 16]):
    ax.plot(GD_METRICS[lr]["total_bits_received"], GD_METRICS[lr]["grad_norms_2"], label="GD", **LINE_KWARGS["Gradient Descent"])
    ax.plot(DRIVE_METRICS[lr]["total_bits_received"], DRIVE_METRICS[lr]["grad_norms_2"], label="MARINA + DRIVE", **LINE_KWARGS["MARINA + DRIVE"])
    ax.plot(CORRELATED_METRICS[lr]["total_bits_received"], CORRELATED_METRICS[lr]["grad_norms_2"], label="MARINA + CQ", **LINE_KWARGS["MARINA + CQ"])
    ax.plot(UNCORRELATED_METRICS[lr]["total_bits_received"], UNCORRELATED_METRICS[lr]["grad_norms_2"], label="MARINA + IQ", **LINE_KWARGS["MARINA + IQ"])
    
    ax.set_yscale("log")
    ax.set_title(f"{lr=}")
    ax.legend()
    
fig.tight_layout()
plt.show()

In [ ]:
GLOBAL_METRICS["splice"] = {
    "Gradient Descent": GD_METRICS[4], 
    "MARINA + DRIVE": DRIVE_METRICS[4],
    "MARINA + CQ": CORRELATED_METRICS[4], 
    "MARINA + IQ": UNCORRELATED_METRICS[8],
}

## Plots

In [ ]:
plt.rcParams["axes.formatter.limits"] = (0, 0)

fig = plot_libsvm(GLOBAL_METRICS)

In [ ]:
fig.savefig("../images/libsvm.pdf", bbox_inches='tight')